# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [3]:
# Run this code
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')


Examine the `salaries` dataset using the `head` function below.

In [5]:
# Your code here

salaries.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [23]:
# Your code here

# Import the ttest_1samp function
from scipy.stats import ttest_1samp

# Filter the data for hourly workers
hourly_workers = salaries[salaries['Full or Part-Time'] == 'P']

# Perform the one-sample t-test
t_stat, p_value = ttest_1samp(hourly_workers['Hourly Rate'].dropna(), 30)

# Output the results
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")


T-statistic: -87.0979044239691
P-value: 0.0


### Challenge 2 - Hypothesis Test Results

In this test, we aimed to determine whether the hourly wage of all hourly workers is significantly different from `$30` per hour. The null hypothesis is that the mean hourly wage is equal to `$30`.

The results of the one-sample t-test are as follows:

- **T-statistic**: -87.10
- **P-value**: 0.0

#### Interpretation:
- **T-statistic**: The t-statistic of -87.10 indicates that the sample mean is far from the hypothesized population mean of `$30/hr`.
- **P-value**: A p-value of 0.0 is much smaller than the commonly used significance level of 0.05. This means that we **reject the null hypothesis**, indicating that the mean hourly wage is significantly different from `$30/hr`.

Therefore, based on this analysis, we conclude that the hourly wage for the workers in this dataset is statistically significantly different from `$30`.


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [31]:
# Your code here

from scipy.stats import t, sem

# Calculate the mean and standard error of the hourly workers' hourly rate
mean_hourly_rate = hourly_workers['Hourly Rate'].dropna().mean()
std_error = sem(hourly_workers['Hourly Rate'].dropna())

# Degrees of freedom (n - 1)
df = len(hourly_workers['Hourly Rate'].dropna()) - 1

# Compute the 95% confidence interval
conf_interval = t.interval(0.95, df, loc=mean_hourly_rate, scale=std_error)

# Output the confidence interval
print(f"95% Confidence Interval: {conf_interval}")


95% Confidence Interval: (16.9035353713793, 17.48030967883591)


### Challenge 3 - Confidence Interval Results

In this challenge, we calculated the 95% confidence interval for the mean hourly wage of all hourly workers. A confidence interval provides a range of values within which we can be confident that the true population mean lies. For a 95% confidence interval, we can be 95% sure that the true mean of the population falls within the calculated range.

The results of the 95% confidence interval are as follows:

- **95% Confidence Interval**: (16.90, 17.48)

#### Interpretation:
- The lower bound of the confidence interval is `$16.90` and the upper bound is `$17.48`.
- This means that we are 95% confident that the true mean hourly wage of all hourly workers in the population lies between `$16.90` and `$17.48`.
- Since this interval does not include `$30`, it reinforces the earlier hypothesis test that the hourly wage is significantly different from `$30/hr`.

This confidence interval provides further evidence that the hourly wage of the workers in the dataset is significantly different from the hypothesized value of `$30`.


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [36]:
# Your code here

from statsmodels.stats.proportion import proportions_ztest

# Calculate the number of hourly workers and total number of workers
num_hourly_workers = len(hourly_workers)
total_workers = len(salaries)

# Hypothesized proportion (25%)
hypothesized_proportion = 0.25

# Perform the proportions z-test
z_stat, p_value = proportions_ztest(num_hourly_workers, total_workers, hypothesized_proportion)

# Output the results
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")


Z-statistic: -140.0706441086974
P-value: 0.0


### Challenge 4 - Hypothesis Test of Proportions Results

In this challenge, we performed a hypothesis test to determine if the proportion of hourly workers in the City of Chicago is significantly different from 25%. The null hypothesis is that the proportion of hourly workers is equal to 25%.

The results of the hypothesis test are as follows:

- **Z-statistic**: -140.07
- **P-value**: 0.0

#### Interpretation:
- **Z-statistic**: The z-statistic of -140.07 is extremely negative, suggesting that the observed proportion of hourly workers is far from the hypothesized proportion of 25%.
- **P-value**: A p-value of 0.0 is much smaller than the commonly used significance level of 0.05. This indicates that we **reject the null hypothesis**, meaning that the proportion of hourly workers is significantly different from 25%.

Thus, based on the hypothesis test, we conclude that the proportion of hourly workers in the City of Chicago is significantly different from 25%.
